In [2]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

In [1]:
#connects to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gradio as gr
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torchvision.transforms.functional as TF
#import cv2
#import matplotlib.pyplot as plt
#from skimage import color
#from skimage import io
#import torchvision.transforms as transforms


In [4]:
class MyModel(nn.Module):
  def __init__(self):
    #fill out
    super(MyModel, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 3)
    #input is 1 28 by 28 matrix
    #output is 5 26 by 26 matrices - - based on convolution math/process/hyperparameters
    self.conv2 = nn.Conv2d(in_channels = 5, out_channels = 3, kernel_size = 4, stride = 2)
    #input is 5 26 by 26 matrices
    #output is 3 12 by 12 matrices  - based on convolution math/process/hyperparameters
    self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding=0)
    #input is 3 12 by 12 matices
    #output is 3 6 by 6 matrices
    #self.linear1 = nn.Linear(3 * 6 * 6, 1000)
    self.linear1 = nn.LazyLinear(1000)
    #input is 3*12*12 = 432 because the convolutional layer before outputs/returns 3 12 by 12 matrices and passes it into linear 1
    # 432 is represented in the 1D shape. the (conv2) output's 1D shape will be [batch, 3*12*12 = 432]. The 1D shape is seen by typing
    self.linear2 = nn.Linear(1000, 500)
    self.linear3 = nn.Linear(500, 250)
    self.linear4 = nn.Linear(250, 125)
    self.linear5 = nn.Linear(125, 10)
    # output is 10 bc it will return 10 values, corresponding to the 10 possible numbers/outputs (from 0 to 9)
    #self.dropout = nn.Dropout(0.2) #dropout is only used during training not evaluation


  def forward(self, images):
    x = self.conv1(images)
    x = self.conv2(x)
    x = self.pool1(x)
    x = x.flatten(start_dim = 1)
    x = torch.relu(self.linear1(x))
    #x = self.dropout(x)
    x = torch.relu(self.linear2(x))
    #x = self.dropout(x)
    x = torch.relu(self.linear3(x))
    #x = self.dropout(x)
    x = torch.relu(self.linear4(x))
    #x = self.dropout(x)
    x = self.linear5(x) #will return 10 values, corresponding to the 10 possible numbers/outputs (from 0 to 9)
    #x = self.dropout(x)
    x = F.softmax(x, dim = 1) #will return 10 probabilties, corresponding to the probabilities of each number
    # when it comes to activation functions that return probabilities: Sigmoid is used for binary classification methods where we only have 2 classes, while SoftMax applies to multiclass problems
    return x

model = MyModel()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
import pickle
import gradio as gr
from torch.utils.data import DataLoader

model_name = "number_predict11_1"
with open("/content/drive/MyDrive/UofT Computer Vision" + "/" + model_name +'.pkl', 'rb') as f:
    model = pickle.load(f)


def predict(image):

    #transform = transforms.Compose([transforms.PILToTensor()])
    #image = transform(image)
    image = TF.to_tensor(image)
    #image = torch.tensor(image)
    value = 2
    Value = torch.tensor(np.array(float(value)))
    #value = TF.to_tensor([2])
    test_data = []
    test_data.append([image, Value])

    #testloader = DataLoader(list(zip(image, value)), batch_size = 1, shuffle = True)
    testloader = DataLoader(test_data, batch_size = 1, shuffle = True)
    for X, y in testloader:
      pred = model(X)
      y_pred2 = torch.argmax(pred, dim = 1)
      y_pred2 = y_pred2.tolist()

      y_preds = torch.argsort(pred, dim = 1, descending = True)
      y_preds = y_preds.squeeze(0)
      #squeeze(0) does this to y_preds tensor: tensor([[indices]]) becomes tensor([indices])
      #the y_preds shape goes from ([1, 10]) to ([10])
      y_preds = y_preds.tolist()
      #therefore y_preds list also has a shape of [10] instead of [1,10] (instead of nested list basically -> [[indices]])
      #how to read shape: ([row, column])


      values, indices = torch.topk(pred[0], 10)
      # from the probability list puts the value(probability) and indice together

      LABELS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
      #list of the possible labels in order (in the order of their respective indice in the probability list)

      #confidences = {LABELS[i]: v.item() for i, v in zip(indices, values) }
      #or
      confidences = {str(i.item()): v.item() for i, v in zip(indices, values)} #since indices and class labels are the same
      # top 10 predictions, as a dictionary whose keys are class labels and whose values are probabilities
      #basically turn zip(indices, values) into a dictionary, where indices will coresspond to the keys/classes/labels and values will be the probabilties

      #first indice in zip(indices, value) is 0, LABELS[0] is '0'
      #second indice in zip(indices, value) is 1, LABELS[1] is '1'
      #indices in zip(indices, value) should correspond to the correct label/indice in the LABELS list

      y = y.tolist()
      #str(y_pred2[0])


      answer =  "our top 3 guesses were " + str(y_preds[0]) + ", " + str(y_preds[1]) + " and " + str(y_preds[2])
    return confidences, answer





gr_inputs = [gr.Sketchpad(label = 'image')]




iface = gr.Interface(fn=predict, inputs= gr_inputs, outputs=["label", "text"])

iface.launch(share=True, show_error=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20d3665474e2910031.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
lt = torch.tensor([[1, 2, 3]])
print(lt)
print(lt.squeeze(0))
print(torch.argsort(lt, descending = True))

tensor([[1, 2, 3]])
tensor([1, 2, 3])
tensor([[2, 1, 0]])
